# ===========================================
# 1_data_exploration.ipynb
# Analyse von generierten Texten & Feedback im Feedback AI-Projekt
# ===========================================

# Zelle 1: Bibliotheken importieren
import os
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
from collections import Counter

# Für Inline-Plots in Jupyter Notebooks
%matplotlib inline

# Stelle sicher, dass NLTK-Daten vorhanden sind
nltk.download('punkt')

# ===========================================
# Zelle 2: Daten einlesen
# ===========================================
# Angenommen, die Feedback-Daten liegen in einer CSV-Datei im Ordner data/raw/
# mit den Spalten "generated_text" und "feedback".
RAW_DATA_DIR = os.path.join("..", "data", "raw")
data_file = os.path.join(RAW_DATA_DIR, "feedback_raw.csv")

df = pd.read_csv(data_file)
print("Anzahl der Einträge:", len(df))
display(df.head())

# ===========================================
# Zelle 3: Datenbereinigung
# ===========================================
def clean_text(text):
    """Entfernt HTML-Tags und überflüssige Leerzeichen, konvertiert den Text in Kleinbuchstaben."""
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip().lower()

df["clean_text"] = df["generated_text"].apply(clean_text)
print("\nBeispiel bereinigter Texte:")
display(df[["generated_text", "clean_text", "feedback"]].head())

# ===========================================
# Zelle 4: Tokenisierung & Wortstatistiken
# ===========================================
df["tokens"] = df["clean_text"].apply(nltk.word_tokenize)

# Erstelle einen Counter für die Token
all_tokens = [token for tokens in df["tokens"] for token in tokens]
token_counts = Counter(all_tokens)

print("\nTop 10 häufigste Wörter:")
print(token_counts.most_common(10))

# Visualisiere die häufigsten Wörter
tokens, freqs = zip(*token_counts.most_common(10))
plt.figure(figsize=(8, 4))
plt.bar(tokens, freqs, color="cornflowerblue")
plt.xlabel("Wörter")
plt.ylabel("Häufigkeit")
plt.title("Top 10 häufigste Wörter in generierten Texten")
plt.xticks(rotation=45)
plt.show()

# ===========================================
# Zelle 5: Feedback-Analyse
# ===========================================
# Analysiere die Verteilung des Feedbacks (z. B. "thumbs_up" vs. "thumbs_down")
feedback_counts = df["feedback"].value_counts()
print("\nFeedback-Verteilung:")
print(feedback_counts)

plt.figure(figsize=(6, 4))
feedback_counts.plot(kind="bar", color=["seagreen", "tomato"])
plt.xlabel("Feedback")
plt.ylabel("Anzahl")
plt.title("Verteilung des Nutzerfeedbacks")
plt.xticks(rotation=0)
plt.show()

# ===========================================
# Zelle 6: Textlängen-Analyse
# ===========================================
df["text_length"] = df["clean_text"].apply(len)
df["token_count"] = df["tokens"].apply(len)

print("\nStatistiken zur Textlänge (Zeichen):")
display(df["text_length"].describe())
print("\nStatistiken zur Wortanzahl (Token):")
display(df["token_count"].describe())

plt.figure(figsize=(10, 4))
plt.hist(df["token_count"], bins=30, color="mediumpurple", alpha=0.8)
plt.xlabel("Anzahl Tokens")
plt.ylabel("Anzahl Texte")
plt.title("Verteilung der Wortanzahl in generierten Texten")
plt.show()

# ===========================================
# Zelle 7: Fazit & Nächste Schritte
# ===========================================
print("""
Fazit:
- Die Daten umfassen generierte Texte mit zugehörigem Nutzerfeedback.
- Die Tokenisierung und Wortfrequenzanalyse zeigen die häufigsten Wörter und geben Aufschluss über den Sprachgebrauch.
- Die Feedback-Verteilung zeigt, ob mehr positive oder negative Bewertungen vorliegen.

Nächste Schritte:
- Weitere Bereinigung und Vorverarbeitung der Daten (siehe Notebook 2_data_preprocessing.ipynb).
- Analyse, ob bestimmte Textlängen oder Wortmuster mit positivem/negativem Feedback korrelieren.
- Vorbereitung der Daten für das Training eines Feedback-gesteuerten Modells.
""")
